<div style="text-align: center">
<b><font size=6>ChEn 3603 Homework 8 Problem 2
    </font></b>
</div>

<div class="alert alert-block alert-danger">

&copy; 2020 This material is copyright protected. Distributing this material in any form without written permission from Professor Sutherland is prohibited and may result in academic discipline.

Here we want to calculate the bubble-point temperature and the composition of the first vapor bubble of an equimolar mixture of ethylene, n-butane, iso-pentane and n-hexane at 4 atm using a few different techniques.

To determine the bubble point temperature, we solve the simplified Rachford-Rice equation, 
\begin{equation}
    \label{eq:bubble-point-eqn}
    r(T) = 1 - \sum_{i=1}^{C} z_{i} K_{i}
\end{equation}
where $z_{i}$ is the original liquid composition.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# import thermo tools
from redlich_kwong_kvalue import redlich_kwong_kvalue
from raoult_law_kvalue import raoult_law_kvalue
from depriester import depriester

In [2]:
P_atm  = 4               # pressure in atm
P_pa   = P_atm * 101325  # pressure in Pa
P_psi  = P_atm * 14.7    # pressure in psia

z = np.ones(4) / 4  # equimolar

# Part 1 - DePriester Correlation

The DePriester correlation is:
$$
    \ln K = \frac{ a_{1} }{ T^{2} } + \frac{a_{2}}{T} + a_{3} + b_{1} \ln p + \frac{b_{2}}{p^{2}} + \frac{b_{3}}{p}
$$
For the system here, the coefficients are summarized in the following table.

|  &#xfeff; | $a_{1}$     | $a_{2}$ | $a_{3}$ | $b_{1}$  | $b_{2}$  | $b_{3}$ |
|-----------|-------------|---------|---------|----------|----------|---------|
| ethylene  | -600076.875 | 0       | 7.90595 | -0.84677 | 42.94594 | 0       |
| n-butane  | -1280557    | 0       | 7.94986 | -0.96455 | 0        | 0       |
| i-pentane | -1481583    | 0       | 7.58071 | -0.93159 | 0        | 0       |
| n-hexane  | -1778901    | 0       | 6.96783 | -0.84634 | 0        | 0       |


In [4]:
# Recall: use T in Rankine and P in psia for DePriester correlation!

dpcoefs = np.array([
  [ -600076.875, 0, 7.90595, -0.84677, 42.94594, 0 ],  # ethylene
  [ -1280557,    0, 7.94986, -0.96455, 0,        0 ],  # n-butane
  [ -1481583,    0, 7.58071, -0.93159, 0,        0 ],  # i-pentane
  [ -1778901,    0, 6.96783, -0.84634, 0,        0 ]   # n-pentane
  ])


# for bubble point:
def depreiester_residual(T,coefs,P):
    return 1 - sum( z * depriester(coefs,T,P) )

Tb = fsolve( depreiester_residual, 400, args=(dpcoefs,P_psi) )
K_depriester = depriester( dpcoefs, Tb[0], P_psi )
y_depriester = K_depriester * z

# convert temperature to K for use later on
T_depriester = Tb[0]*5/9

# print('Bubble point temperature (K): {:.1f}\n'.format(T_depriester) )

# Part 2 - Raoult's Law

For Raoult's law, we have 
\begin{equation}
    \label{eq:Raoult's-law-K}
    K_{i}=\frac{P_{i}^{s}}{P}
\end{equation}
where $P_{i}^{s}$ is the saturation pressure of species $i$ at the system temperature, which we approximate using the Antoine equation, 
\begin{equation}
    \label{eq:antoine}
    \log_{10}P_{i}^{s} = A - \frac{B}{C+T}.
\end{equation}
The parameters for the Antoine equation are given in the following table.

| Species   | A       | B        | C       | 
| --------- | ------- | -------- | ------- |
| ethylene  | 6.74756 | 585.000  | 255.000 | 
| n-butane  | 6.80896 | 935.860  | 238.73  | 
| i-pentane | 6.83315 | 1040.730 | 235.445 | 
| n-hexane  | 6.87024 | 1168.720 | 224.210 | 

To obtain the bubble point, we solve \eqref{eq:bubble-point-eqn} with $K_{i}$ given by \eqref{eq:Raoult's-law-K} and $P_{i}^{s}$ given by [eq:antoine].

In [5]:
antoineCoefs = np.array([
    [6.74756, 585.000,  255.000 ],
    [6.80896, 935.860,  238.73  ],
    [6.83315, 1040.730, 235.445 ],
    [6.87024, 1168.72,  224.210 ]
    ])

def raoult_residual(T,P,coefs):
    K = raoult_law_kvalue(T,P,coefs)
    return 1 - sum(z*K)


# find the bubble point temperature in K
T = fsolve( raoult_residual, T_depriester, args=(P_pa,antoineCoefs) )
T_raoult = T[0]
K_raoult = raoult_law_kvalue(T_raoult,P_pa,antoineCoefs)
y_raoult = K_raoult * z

# print('Bubble point temperature (K): {:.1f}\n'.format(T_raoult) )

# Part 3 - Redlich-Kwong Equation of State

The Redlich-Kwong equation of state uses mixing rules for mixtures that are provided in SHR $\S$2.5.1-2.5.2. 
The python file that I provided on the class web page calculates $K_{i}$ given the critical parameters of each species, the temperature, pressure and composition of the mixture, 
\begin{equation}
    \label{eq:SRK-Kvalue}
    K_{i} = K_{i} \left(T_{c},P_{c},T,P,x_{j},y_{j} \right).
\end{equation}

Since \eqref{eq:SRK-Kvalue} requires the composition, we must employ an iterative procedure to obtain the _K_-values:
 1. Guess $K_{i}$ (e.g., from Raoult's law).
 1. Guess $y_{i} = K_{i}x_{i}$
 1. Solve \eqref{eq:bubble-point-eqn} with \eqref{eq:SRK-Kvalue} for $K_{i}$ to obtain $T$.
 1. Given the new $T$, update $K_{i}$ from \eqref{eq:SRK-Kvalue}
 1. Update $y_{i} = K_i x_i$
 1. If $T$ is changing still, return to step (3). Otherwise, we have the bubble point temperature
 
The following table provides the relevant parameters for the RK (and SRK) equations of state:

| Species   | $T_{c}$ (K) | $P_{c}$ (Pa) | $\omega$ | 
| --------- | ----------- | ------------ | -------- |
| ethylene  | 282.4 | 5.04$\times10^{6}$ | 0.089 | 
| n-butane  | 425.2 | 3.80$\times10^{6}$ | 0.199 | 
| i-pentane | 460.4 | 3.39$\times10^{6}$ | 0.227 | 
| n-hexane  | 507.5 | 3.01$\times10^{6}$ | 0.299 | 


In [6]:
def rk_residual(T,P,Tc,Pc,x,y):
    K = redlich_kwong_kvalue(Tc,Pc,T,P,x,y)
    return 1 - sum( z * K )

Tc = np.array([ 282.4, 425.2, 460.4, 507.5 ])
Pc = np.array([ 5.04,  3.80,  3.39,  3.01  ]) * 1e6

K_rk = K_raoult
y_rk = y_raoult
T_rk = T_raoult
err  = 1

while err>0.01:
    T = fsolve( rk_residual, T_rk, args=(P_pa,Tc,Pc,z,y_rk) )
    err = abs(T[0] - T_rk)
    # update temperature and composition
    T_rk = T[0]
    K_rk = redlich_kwong_kvalue(Tc,Pc,T_rk,P_pa,z,y_rk)
    y_rk = K_rk * z
    
# print('Bubble point temperature (K): {:.1f}\n'.format(T_rk) )

In [10]:
print(Tc,Pc,P_pa)
print(redlich_kwong_kvalue(Tc,Pc,240,P_pa,z,y_rk))

[282.4 425.2 460.4 507.5] [5040000. 3800000. 3390000. 3010000.] 405300
[3.89109547 0.11336971 0.03992669 0.00954489]


# Part 4 - Soave-Redlich-Kwong

The SRK equation of state follows the same approach as the RK equation of state, with incorporation of the acentric factor, $\omega$, which is given in the table above.

In [8]:
def srk_residual(T,P,Tc,Pc,x,y,w):
    K = redlich_kwong_kvalue(Tc,Pc,T,P,x,y,w)
    return 1 - sum( z * K )

# define the acentric factor for each species
w = np.array([ 0.089, 0.199, 0.227, 0.299 ])

T_srk = T_rk
K_srk = K_rk
y_srk = y_rk
err   = 1

while err>0.01:
    T = fsolve( srk_residual, T_srk, args=(P_pa,Tc,Pc,z,y_srk,w) )
    err = abs(T[0] - T_srk)
    # update temperature and composition
    T_srk = T[0]
    K_srk = redlich_kwong_kvalue(Tc,Pc,T_srk,P_pa,z,y_srk,w)
    y_srk = K_srk * z

# print('Bubble point temperature (K): {:.1f}\n'.format(T_srk) )

# Summary of Results

The results for the bubble point calculation showing the temperature and composition of the vapor are summarized in the table below.

In [9]:
print('-----------------------------------------------------------')
print('Method      T (K)  Ethylene  n-Butane  i-Pentane  n-Pentane')
print('-----------------------------------------------------------')
print('DePriester {:6.1f} {:9.3f} {:9.3f} {:10.3f} {:10.3f}'.format(T_depriester,y_depriester[0],y_depriester[1],y_depriester[2],y_depriester[3]))
print('Raoult     {:6.1f} {:9.3f} {:9.3f} {:10.3f} {:10.3f}'.format(T_raoult,y_raoult[0],y_raoult[1],y_raoult[2],y_raoult[3]))
print('RK         {:6.1f} {:9.3f} {:9.3f} {:10.3f} {:10.3f}'.format(T_rk,y_rk[0],y_rk[1],y_rk[2],y_rk[3]))
print('SRK        {:6.1f} {:9.3f} {:9.3f} {:10.3f} {:10.3f}'.format(T_srk,y_srk[0],y_srk[1],y_srk[2],y_srk[3]))
print('-----------------------------------------------------------')

-----------------------------------------------------------
Method      T (K)  Ethylene  n-Butane  i-Pentane  n-Pentane
-----------------------------------------------------------
DePriester  244.2     0.976     0.018      0.005      0.001
Raoult      237.3     0.983     0.013      0.003      0.000
RK          239.4     0.960     0.028      0.010      0.002
SRK         240.3     0.977     0.017      0.005      0.001
-----------------------------------------------------------


It is interesting to note that there is significant discrepancy between the various methods - particularly for temperature.  I'd be inclined to trust the RK/SRK results over the DePriester and Raoult's law results.